# Launch Montitors
Launch monitors can measure a variety of parameters, including:

- Ball Speed: The speed of the golf ball immediately after impact.
- Launch Angle: The angle at which the ball leaves the clubface relative to the ground.
- Spin Rate: The rate at which the ball spins around its axis.
- Carry Distance: The distance the ball travels through the air before hitting the ground.
- Total Distance: The total distance the ball travels, including roll.
- Club Speed: The speed of the clubhead at impact.
- Smash Factor: The ratio of ball speed to club speed, indicating the efficiency of the impact.

# Calcs
- MPH to MPS: 1 mph = 1/2.2375 = 0.44704 m/s
- Total Spin: total spin * math(cos(math.radians(spin_axis)))

## MLM2Pro GSPro Connector
- https://github.com/springbok/MLM2PRO-GSPro-Connector
- Screen scrapes data from the UI in practice mode via a window on the AirPlay (iPhone)
- Uses ROI (Regions of Interest) to "screen scrape" the data
- Converts MPH to MPS (meters per second) with 2.2375 (although the approx is 2.23693629)
    - Ball speed: 150 mph (=150/10.0*2.2375) 33.5625? but this is 67.056 mps
- [GSPro Open API](https://gsprogolf.com/GSProConnectV1.html) 
```json
{
    "DeviceID": "GSPro LM 1.1",  			//required - unqiue per launch monitor / prooject type
    "Units": "Yards",						//default yards
    "ShotNumber": 13,						//required - auto increment from LM
    "APIversion": "1",						//required - "1" is current version
    "BallData": {		
        "Speed": 147.5,						//required
        "SpinAxis": -13.2,					//required
        "TotalSpin": 3250.0,				//required
        "BackSpin": 2500.0,					//only required if total spin is not sent
        "SideSpin": -800.0,					//only required if total spin is not sent
        "HLA": 2.3,							//required
        "VLA": 14.3,						//required
        "CarryDistance": 256.5				//optional
    },
    "ClubData": {							
        "Speed": 0.0,
        "AngleOfAttack": 0.0,
        "FaceToTarget": 0.0,
        "Lie": 0.0,
        "Loft": 0.0,
        "Path": 0.0,
        "SpeedAtImpact": 0.0,
        "VerticalFaceImpact": 0.0,
        "HorizontalFaceImpact": 0.0,
        "ClosureRate": 0.0
    },
    "ShotDataOptions": {
        "ContainsBallData": true,			//required
        "ContainsClubData": false,			//required
        "LaunchMonitorIsReady": true, 		//not required
        "LaunchMonitorBallDetected": true, 	//not required
        "IsHeartBeat": false 				//not required
    }
}
```


# Calculations
- [What A Drag - Projectile Motion with Drag in Excel](https://www.youtube.com/watch?v=ShWSK1YF61Q)
- https://www.youtube.com/watch?v=k_ttiagCNUQ
`distance = (initial_velocity^2 * sin(2 * launch_angle)) / gravity`

launch_angle has to be represented in radians in most tech including Excel and Python

## Excel
|Detail|Value|
|--|--|
Velocity (mph)|150
Velocity (mps)|=B1/2.23694


B1=velocity in MPH
B2=velocity in MPS (=B1/2.23694)
B3=Launch Angle (Degrees)
B4=Launch Angle (Radians) (=RADIANS(B3))

In [3]:
import math

# Constants
MPS_TO_MPH = 2.23694
GRAVITATIONAL_ACCELERATION = 9.80665
METERS_TO_YARDS = 1.09361
DRAG_COEFFICIENT = 0.7

In [5]:
# Example usage
ball_speed_mph = 141  # miles per hour
ball_speed_mps = ball_speed_mph / MPS_TO_MPH  # Convert to meters per second
launch_angle = 25  # degrees
spin_rate_mph = 3000  # RPM (not used in this simplified calculation)

Vo=math.radians(launch_angle)
Vox=ball_speed_mps*math.cos(Vo)
print("Vox: ", Vox)
# Print in yards
print("Vox in yards: ", Vox * METERS_TO_YARDS)

Vox:  57.12687777596522
Vox in yards:  62.47452480457332


In [22]:
# Information GSPro needs
ballData = {    
    "Speed": 147.5,						# required
    "SpinAxis": -13.2,					# required
    "TotalSpin": 1088.0,				# required
    "BackSpin": 2500.0,					# only required if total spin is not sent
    "SideSpin": -800.0,					# only required if total spin is not sent
    "HLA": 2.3,							# required
    "VLA": 14.3,						# required
    "CarryDistance": 0,			# optional
}

ballData.update({"Speed": 142/2.2375})
ballData.update({"SpinAxis": -5.0})
ballData.update({"VLA": 18.9})
ballData.update({"HLA": 4.6})

ball_speed=142/2.2375 #mph
speed=142/10.0*2.2375 #m/s
club_speed=101.2/MPS_TO_MPH
total_spin=1088 #rpm
spin_axis=-5.0 #degrees
vla=18.9 #vertical launch angle
hla=4.6 #horizontal launch angle
# smash_factor_mlm=1.4
# smash_factor=math.ceil((ball_speed / club_speed)*10)/10
# print("Smash factor: ", smash_factor, "MLM: ", smash_factor_mlm)

print("radians: ", math.radians(ballData["SpinAxis"]))
back_spin=round(ballData["TotalSpin"]*math.cos(math.radians(ballData["SpinAxis"])))
ballData.update({"BackSpin": back_spin})
print("Back spin: ", back_spin)

side_spin=round(ballData["TotalSpin"]*math.sin(math.radians(ballData["SpinAxis"])))
ballData.update({"SideSpin": side_spin})
print("Side spin: ", side_spin)


# Calculate the horizontal velocity component
# Vox = ball_speed_mps * math.cos(math.radians(vla))
# print("Vox: ", Vox)
print(ballData)


radians:  -0.08726646259971647
Back spin:  1084
Side spin:  -95
{'Speed': 63.46368715083799, 'SpinAxis': -5.0, 'TotalSpin': 1088.0, 'BackSpin': 1084, 'SideSpin': -95, 'HLA': 4.6, 'VLA': 18.9}


In [34]:
def calculate_golf_ball_distance(ball_speed, launch_angle):
    """
    Calculate the distance a golf ball travels given the ball speed, launch angle, and spin rate.

    Parameters:
    ball_speed (float): The speed of the golf ball in meters per second.
    launch_angle (float): The launch angle of the golf ball in degrees.

    Returns:
    float: The distance the golf ball travels in meters.
    """
    # Convert launch angle from degrees to radians
    launch_angle_rad = math.radians(launch_angle)
    
    # Acceleration due to gravity (m/s^2)
    g = 9.81
    
    # Simplified calculation of distance (ignoring spin rate for simplicity)
    distance_meters = (ball_speed ** 2) * math.sin(2 * launch_angle_rad) / g
    
    # Convert distance from meters to yards
    distance_yards = abs(distance_meters) * 1.09361
    
    return distance_yards

def calculate_golf_ball_distance_advanced(initial_velocity, launch_angle, spin_rate, altitude=0, temperature=20, humidity=50):
    """
    Calculate the distance a golf ball travels given the initial velocity, launch angle, and spin rate.

    Parameters:
    initial_velocity (float): The initial velocity of the golf ball in meters per second.
    launch_angle (float): The launch angle of the golf ball in degrees.
    spin_rate (float): The spin rate of the golf ball in revolutions per minute (RPM).
    altitude (float): The altitude in meters (default is 0).
    temperature (float): The temperature in degrees Celsius (default is 20).
    humidity (float): The relative humidity in percentage (default is 50).

    Returns:
    float: The distance the golf ball travels in yards.
    """
    # Convert launch angle from degrees to radians
    launch_angle_rad = math.radians(launch_angle)
    
    # Acceleration due to gravity (m/s^2)
    g = 9.81
    
    # Convert spin rate from RPM to radians per second
    spin_rate_rad = spin_rate * 2 * math.pi / 60
    
    # Air density (kg/m^3) - simplified model
    air_density = 1.225 * (1 - 0.0000225577 * altitude) * (273 / (273 + temperature)) * (1 - 0.378 * humidity / 100)
    
    # Drag coefficient (simplified model)
    drag_coefficient = 0.3
    
    # Lift coefficient (simplified model)
    lift_coefficient = 0.2 * spin_rate_rad / initial_velocity
    
    # Calculate the drag and lift forces
    drag_force = 0.5 * air_density * drag_coefficient * initial_velocity**2
    lift_force = 0.5 * air_density * lift_coefficient * initial_velocity**2
    
    # Calculate the effective acceleration due to drag and lift
    effective_acceleration = g - (lift_force / initial_velocity) + (drag_force / initial_velocity)
    
    # Calculate the range using the projectile motion formula with effective acceleration
    distance_meters = (initial_velocity ** 2) * math.sin(2 * launch_angle_rad) / effective_acceleration
    
    # Convert distance from meters to yards
    distance_yards = abs(distance_meters) * 1.09361
    
    return distance_yards

def calculate_golf_ball_distance_with_side_spin(ball_speed, launch_angle, back_spin, side_spin, altitude=0, temperature=20, humidity=50):
    """
    Calculate the distance a golf ball travels given the ball speed, launch angle, back spin, and side spin.

    Parameters:
    ball_speed (float): The speed of the golf ball in meters per second.
    launch_angle (float): The launch angle of the golf ball in degrees.
    back_spin (float): The back spin of the golf ball in revolutions per minute (RPM).
    side_spin (float): The side spin of the golf ball in revolutions per minute (RPM).
    altitude (float): The altitude in meters (default is 0).
    temperature (float): The temperature in degrees Celsius (default is 20).
    humidity (float): The relative humidity in percentage (default is 50).

    Returns:
    float: The distance the golf ball travels in meters.
    """
    # Convert launch angle from degrees to radians
    launch_angle_rad = math.radians(launch_angle)
    
    # Acceleration due to gravity (m/s^2)
    g = 9.81
    
    # Convert spin rates from RPM to radians per second
    back_spin_rad = back_spin * 2 * math.pi / 60
    side_spin_rad = side_spin * 2 * math.pi / 60
    
    # Air density (kg/m^3) - simplified model
    air_density = 1.225 * (1 - 0.0000225577 * altitude) * (273 / (273 + temperature)) * (1 - 0.378 * humidity / 100)
    
    # Drag coefficient (simplified model)
    drag_coefficient = 0.3
    
    # Lift coefficient (simplified model)
    lift_coefficient = 0.2 * back_spin_rad / ball_speed
    
    # Side force coefficient (simplified model)
    side_force_coefficient = 0.2 * side_spin_rad / ball_speed
    
    # Calculate the drag, lift, and side forces
    drag_force = 0.5 * air_density * drag_coefficient * ball_speed**2
    lift_force = 0.5 * air_density * lift_coefficient * ball_speed**2
    side_force = 0.5 * air_density * side_force_coefficient * ball_speed**2
    
    # Calculate the effective acceleration due to drag, lift, and side forces
    effective_acceleration = g - (lift_force / ball_speed) + (drag_force / ball_speed)
    
    # Calculate the range using the projectile motion formula with effective acceleration
    distance_meters = (ball_speed ** 2) * math.sin(2 * launch_angle_rad) / effective_acceleration
    
    # Ensure the distance is positive
    distance_meters = abs(distance_meters)
    
    # Adjust distance for side spin (simplified model)
    distance_meters -= side_force / g
    

    distance_yards = distance_meters * METERS_TO_YARDS
    return distance_yards
def calculate_golf_ball_distance_with_spin_axis(ball_speed, launch_angle, spin_rate, spin_axis, altitude=0, temperature=20, humidity=50):
    """
    Calculate the distance a golf ball travels given the ball speed, launch angle, spin rate, and spin axis.

    Parameters:
    ball_speed (float): The speed of the golf ball in meters per second.
    launch_angle (float): The launch angle of the golf ball in degrees.
    spin_rate (float): The total spin rate of the golf ball in revolutions per minute (RPM).
    spin_axis (float): The spin axis angle in degrees (negative to left, positive to right).
    altitude (float): The altitude in meters (default is 0).
    temperature (float): The temperature in degrees Celsius (default is 20).
    humidity (float): The relative humidity in percentage (default is 50).

    Returns:
    float: The distance the golf ball travels in meters.
    """
    # Convert launch angle and spin axis from degrees to radians
    launch_angle_rad = math.radians(launch_angle)
    spin_axis_rad = math.radians(spin_axis)
    
    # Acceleration due to gravity (m/s^2)
    g = 9.81
    
    # Convert spin rate from RPM to radians per second
    spin_rate_rad = spin_rate * 2 * math.pi / 60
    
    # Decompose spin rate into backspin and sidespin components
    back_spin_rad = spin_rate_rad * math.cos(spin_axis_rad)
    side_spin_rad = spin_rate_rad * math.sin(spin_axis_rad)
    
    # Air density (kg/m^3) - simplified model
    air_density = 1.225 * (1 - 0.0000225577 * altitude) * (273 / (273 + temperature)) * (1 - 0.378 * humidity / 100)
    
    # Drag coefficient (simplified model)
    drag_coefficient = 0.3
    
    # Lift coefficient (simplified model)
    lift_coefficient = 0.2 * back_spin_rad / ball_speed
    
    # Side force coefficient (simplified model)
    side_force_coefficient = 0.2 * side_spin_rad / ball_speed
    
    # Calculate the drag, lift, and side forces
    drag_force = 0.5 * air_density * drag_coefficient * ball_speed**2
    lift_force = 0.5 * air_density * lift_coefficient * ball_speed**2
    side_force = 0.5 * air_density * side_force_coefficient * ball_speed**2
    
    # Calculate the effective acceleration due to drag, lift, and side forces
    effective_acceleration = g - (lift_force / ball_speed) + (drag_force / ball_speed)
    
    # Calculate the range using the projectile motion formula with effective acceleration
    distance_meters = (ball_speed ** 2) * math.sin(2 * launch_angle_rad) / effective_acceleration
    
    # Ensure the distance is positive
    distance_meters = abs(distance_meters)
    
    # Adjust distance for side spin (simplified model)
    distance_meters -= side_force / g

    distance_yards = distance_meters * METERS_TO_YARDS
    
    return distance_yards

def calculate_golf_ball_distance_with_launch_direction(ball_speed, launch_angle, spin_rate, spin_axis, launch_direction, altitude=0, temperature=20, humidity=50):
    """
    Calculate the distance a golf ball travels given the ball speed, launch angle, spin rate, spin axis, and launch direction.

    Parameters:
    ball_speed (float): The speed of the golf ball in meters per second.
    launch_angle (float): The launch angle of the golf ball in degrees.
    spin_rate (float): The total spin rate of the golf ball in revolutions per minute (RPM).
    spin_axis (float): The spin axis angle in degrees (negative to left, positive to right).
    launch_direction (float): The horizontal launch direction angle in degrees (negative to left, positive to right).
    altitude (float): The altitude in meters (default is 0).
    temperature (float): The temperature in degrees Celsius (default is 20).
    humidity (float): The relative humidity in percentage (default is 50).

    Returns:
    float: The distance the golf ball travels in meters.
    """
    # Convert angles from degrees to radians
    launch_angle_rad = math.radians(launch_angle)
    spin_axis_rad = math.radians(spin_axis)
    launch_direction_rad = math.radians(launch_direction)
    
    # Acceleration due to gravity (m/s^2)
    g = 9.81
    
    # Convert spin rate from RPM to radians per second
    spin_rate_rad = spin_rate * 2 * math.pi / 60
    
    # Decompose spin rate into backspin and sidespin components
    back_spin_rad = spin_rate_rad * math.cos(spin_axis_rad)
    side_spin_rad = spin_rate_rad * math.sin(spin_axis_rad)
    
    # Air density (kg/m^3) - simplified model
    air_density = 1.225 * (1 - 0.0000225577 * altitude) * (273 / (273 + temperature)) * (1 - 0.378 * humidity / 100)
    
    # Drag coefficient (simplified model)
    drag_coefficient = 0.3
    
    # Lift coefficient (simplified model)
    lift_coefficient = 0.2 * back_spin_rad / ball_speed
    
    # Side force coefficient (simplified model)
    side_force_coefficient = 0.2 * side_spin_rad / ball_speed
    
    # Calculate the drag, lift, and side forces
    drag_force = 0.5 * air_density * drag_coefficient * ball_speed**2
    lift_force = 0.5 * air_density * lift_coefficient * ball_speed**2
    side_force = 0.5 * air_density * side_force_coefficient * ball_speed**2
    
    # Calculate the effective acceleration due to drag, lift, and side forces
    effective_acceleration = g - (lift_force / ball_speed) + (drag_force / ball_speed)
    
    # Decompose initial velocity into horizontal and vertical components
    initial_velocity_x = ball_speed * math.cos(launch_angle_rad) * math.cos(launch_direction_rad)
    initial_velocity_y = ball_speed * math.sin(launch_angle_rad)
    initial_velocity_z = ball_speed * math.cos(launch_angle_rad) * math.sin(launch_direction_rad)
    
    # Calculate the range using the projectile motion formula with effective acceleration
    distance_meters = (initial_velocity_x ** 2) * math.sin(2 * launch_angle_rad) / effective_acceleration
    
    # Ensure the distance is positive
    distance_meters = abs(distance_meters)
    
    # Adjust distance for side spin (simplified model)
    distance_meters -= side_force / g

    distance_yards = distance_meters * METERS_TO_YARDS
    
    return distance_yards

def calculate_air_density(elevation_ft, temperature_c=15):
    """
    Calculate the air density at a given elevation.

    Parameters:
    elevation_ft (float): The elevation in feet.
    temperature_c (float): The temperature in degrees Celsius (default is 15°C).

    Returns:
    float: The air density in kg/m^3.
    """
    # Convert elevation from feet to meters
    elevation_m = elevation_ft * 0.3048
    
    # Constants
    T0 = 288.15  # Standard temperature at sea level in Kelvin
    P0 = 101325  # Standard pressure at sea level in Pascals
    g = 9.80665  # Gravitational acceleration in m/s^2
    R = 287.05  # Specific gas constant for dry air in J/(kg·K)
    L = 0.0065  # Temperature lapse rate in K/m
    
    # Convert temperature from Celsius to Kelvin
    temperature_k = temperature_c + 273.15
    
    # Calculate the pressure at the given elevation using the barometric formula
    pressure = P0 * (1 - (L * elevation_m) / T0) ** (g / (R * L))
    
    # Calculate the air density using the ideal gas law
    air_density = pressure / (R * temperature_k)
    
    return air_density

# Example usage
elevation_ft = 1200  # feet
temperature_c = 15  # degrees Celsius

air_density = calculate_air_density(elevation_ft, temperature_c)
print(f"The air density at {elevation_ft} feet is {air_density:.4f} kg/m^3.")

# Example usage

air_density = 1.225  # kg/m^3 at sea level and standard conditions

# temp_fahrenheit = 70  # degrees Fahrenheit
# temp_celsius = (temp_fahrenheit - 32) * 5/9
temp_celsius = 20  # degrees Celsius
temp_fahrenheit = (temp_celsius * 9/5) + 32
print("Temperature in Celsius: ", temp_celsius)

def run_test(ball_speed_mph,launch_angle,spin_rate_rpm,launch_direction,spin_axis):
    ball_speed = ball_speed_mph / MPS_TO_MPH  # Convert to meters per second

    distance = calculate_golf_ball_distance(ball_speed, launch_angle)
    # distance_yards = distance * 1.09361

    print(f"Simple: {distance:.2f} yards.")

    # Example usage
    distance = calculate_golf_ball_distance_advanced(ball_speed, launch_angle, spin_rate_rpm)
    print(f"With Spin RPM: {distance:.2f} yards.")

    distance = calculate_golf_ball_distance_with_spin_axis(ball_speed, launch_angle, spin_rate_rpm, spin_axis)
    print(f"With Spin RPM and Axis: {distance:.2f} yards.")

    distance = calculate_golf_ball_distance_with_launch_direction(ball_speed, launch_angle, spin_rate_rpm, spin_axis, launch_direction)
    print(f"With Spin RPM, Axis and Launch Direction: {distance:.2f} yards.")

ball_speed_mph = 142  # miles per hour
ball_speed = ball_speed_mph / MPS_TO_MPH  # Convert to meters per second
launch_angle = 18.9  # degrees
spin_rate_rpm = 1088  # RPM (not used in this simplified calculation)

spin_axis=5.0 # degress absolute
launch_direction=4.6 # degrees absolute

run_test(142,18.9,1088,4.6,5.0)
# print a line 80 dashes
print("-"*80)
print("This is where it doesn't make sense...need some adjustments")
run_test(138,17.1,12157,6.4,3.3)


The air density at 1200 feet is 1.1728 kg/m^3.
Temperature in Celsius:  20
Simple: 275.33 yards.
With Spin RPM: 334.38 yards.
With Spin RPM and Axis: 326.23 yards.
With Spin RPM, Axis and Launch Direction: 289.40 yards.
--------------------------------------------------------------------------------
This is where it doesn't make sense...need some adjustments
Simple: 238.48 yards.
With Spin RPM: 23.52 yards.
With Spin RPM and Axis: -23.09 yards.
With Spin RPM, Axis and Launch Direction: -25.39 yards.


In [11]:
def calculate_ball_speed(distance_yards, launch_angle):
    """
    Calculate the initial velocity (ball speed) given the distance in yards and launch angle.

    Parameters:
    distance_yards (float): The distance the golf ball travels in yards.
    launch_angle (float): The launch angle of the golf ball in degrees.

    Returns:
    float: The initial velocity (ball speed) in meters per second.
    """
    # Convert distance from yards to meters
    distance_meters = distance_yards / 1.09361
    
    # Convert launch angle from degrees to radians
    launch_angle_rad = math.radians(launch_angle)
    
    # Acceleration due to gravity (m/s^2)
    g = 9.81
    
    # Calculate the initial velocity using the rearranged projectile motion formula
    initial_velocity = math.sqrt(distance_meters * g / math.sin(2 * launch_angle_rad))
    
    return initial_velocity

# Example usage
distance_yards = 278.6977573904179  # yards
launch_angle = 45  # degrees

ball_speed = calculate_ball_speed(distance_yards, launch_angle)
print(f"The ball speed is {ball_speed:.2f} meters per second.")

The ball speed is 50.00 meters per second.


In [14]:
# https://www.mycompiler.io/view/3Ta6UVuc0pY
# Given values
ball_speed_mph=142
launch_angle=18.9
v0 = ball_speed_mph/MPS_TO_MPH
# v0 = 48.2  # m/s
# theta = 24.2  # degrees
theta = launch_angle
m = 0.04593  # kg
A = 0.0043  # m^2
Cd = 0.3
Cl = 0.4
g = 9.8  # m/s^2
rho = 1.225  # kg/m^3 (standard air density at sea level)

# Convert angle to radians
theta_rad = math.radians(theta)

# Calculate D
D = (v0**2 * math.sin(2 * theta_rad)) / g + (2 * v0**2 / g) * (m / (rho * A * Cd))**0.5 * math.atan(math.sin(theta_rad) / (math.cos(theta_rad) + (2 * v0 / g) * (m / (rho * A * Cl))**0.5))

print(f"The total distance traveled by the golf ball is approximately {D:.2f} meters.")
# Convert distance from meters to yards
distance_yards = D * 1.09361
print(f"The total distance traveled by the golf ball is approximately {distance_yards:.2f} yards.")

The total distance traveled by the golf ball is approximately 275.40 meters.
The total distance traveled by the golf ball is approximately 301.18 yards.
